In [1]:
import os
os.chdir('../')

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from src.XAI.concepts import concept_instances

In [3]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

In [3]:
concept_cavs = {}
for concept in concept_instances.values():
    probe, score = concept.load_torch_probe()
    if score > 0.5:
        cav = probe[1].weight.data.cpu().numpy().squeeze()
        concept_cavs[concept.name] = cav
    print(f"{concept.name:^35} | {score:^10.4f}")

          player to right           |   0.8810  
            player low              |   0.9890  
            difficulty              |   0.8040  
           player in air            |   0.9650  
          player falling            |   0.9510  
       player dodging in air        |   0.9480  
       player fall velocity         |   0.7830  
      player standing on wall       |   0.9250  
      player dodging on wall        |   0.9450  
          events quantity           |   0.7650  
           visible wall             |   0.8960  
          visible bullet            |   0.9080  
           lava quantity            |   0.8820  
           coin quantity            |   0.6690  
        good visible events         |   0.7300  
        bad visible events          |   0.8420  
         two close bullets          |   0.0000  
      bullet player distance        |   0.7700  
    bullet aligned with player      |   0.6870  
          player dodging            |   0.9900  


In [4]:
num_concepts = len(concept_cavs)
similarity_matrix = np.zeros((num_concepts, num_concepts))

# Populate the matrix
for i, concept1 in enumerate(concept_cavs):
    for j, concept2 in enumerate(concept_cavs):
        similarity_matrix[i, j] = cosine_similarity(concept_cavs[concept1], concept_cavs[concept2])

plt.figure(figsize=(10, 8))
plt.imshow(similarity_matrix, cmap='coolwarm', interpolation='nearest')
# set x and y labels
plt.xlabel('Concept')
plt.ylabel('Concept')
cbar = plt.colorbar()
cbar.set_label('Cosine Similarity', rotation=270, labelpad=15)
plt.title('Concept Cosine Similarity Matrix')
plt.xticks(np.arange(num_concepts), concept_cavs.keys(), rotation=90)
plt.yticks(np.arange(num_concepts), concept_cavs.keys())
plt.tight_layout()
plt.savefig('figures/concept_similarity.png', dpi=300)
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.01792368 -0.01672214 -0.0476399  ...  0.06584161  0.0189722
 -0.0184699 ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
pairs_with_scores = []
for i, concept1 in enumerate(concept_cavs.keys()):
    for j, concept2 in enumerate(concept_cavs.keys()):
        if i < j:  # No duplicate pairs
            similarity = cosine_similarity(concept_cavs[concept1], concept_cavs[concept2])
            pairs_with_scores.append(((concept1, concept2), similarity))

# Sort pairs based on similarity
sorted_pairs = sorted(pairs_with_scores, key=lambda x: x[1], reverse=True)

# Extract top x and bottom x correlated concept pairs
x = 10
top_x_concepts = sorted_pairs[:x]
bottom_x_concepts = sorted_pairs[-x:]

print(f"Top {x} Correlated Concepts:\n")
print(f"{'Concept 1':^40} | {'Concept 2':^40} | {'Score':^10}")
print('-' * 100)
for (pair, score) in top_x_concepts:
    print(f"{pair[0]:^40} | {pair[1]:^40} | {score:^10.4f} |")

print(f"\nBottom {x} Correlated Concepts:\n")
print(f"{'Concept 1':^40} | {'Concept 2':^40} | {'Score':^10}")
print('-' * 100)
for (pair, score) in bottom_x_concepts:
    print(f"{pair[0]:^40} | {pair[1]:^40} | {score:^10.4f} |")
    

Top 10 Correlated Concepts:

               Concept 1                 |                Concept 2                 |   Score   
----------------------------------------------------------------------------------------------------
             player in air               |              player falling              |   0.7488   |
         player dodging in air           |              player dodging              |   0.6629   |
             coin quantity               |           good visible events            |   0.6294   |
            events quantity              |              coin quantity               |   0.6130   |
            events quantity              |           good visible events            |   0.6019   |
             visible bullet              |        bullet aligned with player        |   0.5659   |
            events quantity              |               visible wall               |   0.4651   |
               player low                |              player dodging          